In [17]:
nodes = []


In [18]:
class Graph:
    def __init__(self):
        self.in_edges = []
        self.out_edges = []
        self.light_combinations = {}
        self.incoming_congestion = {}

    def add_inEdge(self, index):
        self.in_edges.append(index)
    
    def add_outEdge(self, index):
        self.out_edges.append(index)
    
    def addEdge(self, index1, index2, light_combination):
        self.in_edges.append(index1)
        self.out_edges.append(index1)
        nodes[index1].in_edges.append(index2)
        nodes[index1].out_edges.append(index2)
        self.light_combinations[(index1, index2)] = light_combination
        self.light_combinations[(index2, index1)] = light_combination  # Assuming bidirectional roads have same light combination

    def update_incoming_congestion(self, incoming_edge, congestion_level):
        self.incoming_congestion[incoming_edge] = congestion_level

In [19]:
for i in range(0,5):
    g = Graph()
    nodes.append(g)

In [20]:
#example case with 5 nodes

nodes[0].addEdge(1, 0, {"1-4": "Green", "4-1": "Green", "1-5": "Red"})
nodes[0].update_incoming_congestion(1, 20)  # Example congestion level
nodes[1].addEdge(2, 1, {"2-1": "Green", "1-2": "Green", "2-4": "Red"})
nodes[1].update_incoming_congestion(0, 15)  # Example congestion level
nodes[2].addEdge(3, 2, {"3-2": "Green", "2-3": "Green", "3-5": "Red"})
nodes[2].update_incoming_congestion(1, 25)  # Example congestion level
nodes[3].addEdge(4, 3, {"4-1": "Green", "1-4": "Green", "4-3": "Red"})
nodes[3].update_incoming_congestion(2, 30)  # Example congestion level
nodes[4].addEdge(0, 4, {"5-1": "Red", "1-5": "Red", "5-3": "Green"})
nodes[4].update_incoming_congestion(3, 18)


In [21]:
for node in nodes:
    for neighbour in node.in_edges:
        print(neighbour,end = " ")
    print()

1 2 
0 3 4 
0 3 5 
1 2 4 6 
1 3 7 
2 6 
3 5 7 
4 6 


In [ ]:
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

In [ ]:
# Prepare input data
input_features = []
for node in nodes:
    features = [node.incoming_congestion[edge] for edge in node.in_edges]
    input_features.append(features)


In [ ]:
# Prepare output data (target traffic light combinations)
output_labels = []
for node in nodes:
    labels = [node.light_combinations[edge] for edge in node.out_edges]
    output_labels.append(labels)


In [ ]:
# Encode output labels
label_encoder = LabelEncoder()
encoded_labels = [label_encoder.fit_transform(labels) for labels in output_labels]


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(input_features, encoded_labels, test_size=0.2, random_state=42)

In [ ]:
# Build neural network model
model = Sequential([
    Dense(64, activation='relu', input_shape=(len(X_train[0]),)),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dense(len(label_encoder.classes_), activation='softmax')
])


In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(np.array(X_train), np.array(y_train), epochs=50, batch_size=32, validation_data=(np.array(X_test), np.array(y_test)))

In [ ]:
new_scenario = [[25, 20, 30, 15], [20, 25, 18], [30, 15, 25], [18, 30, 20], [15, 20, 25]]
predicted_labels = model.predict(np.array(new_scenario))
predicted_combinations = [label_encoder.classes_[np.argmax(labels)] for labels in predicted_labels]
print("Predicted Traffic Light Combinations:")
for i, combination in enumerate(predicted_combinations):
    print(f"Node {i+1}: {combination}")